> Code run locally due to server at 3.11 which is not supported by PyMOL



In [14]:
import os
from pathlib import Path
root = Path('/Users/user/Coding/EV-D68-3C-protease/')
os.chdir(root)

In [15]:
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser
import pandas as pd
import ipywidgets as wd


metadata: pd.DataFrame = pd.read_csv('metadata-from-Fragalysis.csv',
                                     index_col=0)




PandasTools.AddMoleculeColumnToFrame(metadata, smilesCol='new_smiles', molCol='mol')
out = wd.Output()
with out:  # suppress output the crappy way
    metadata['single_smiles'] = metadata.mol.apply(LargestFragmentChooser(preferOrganic=True).choose).apply(Chem.MolToSmiles)

metadata.iloc[0]

crystal_name                                D68EV3CPROA-x0102_0B
RealCrystalName                                D68EV3CPROA-x0102
smiles                                            Cc1ccsc1C(=O)O
new_smiles                                        Cc1ccsc1C(=O)O
alternate_name                                        Z104474228
site_name                                                      5
pdb_entry                                                    NaN
mol                <rdkit.Chem.rdchem.Mol object at 0x163d2c200>
single_smiles                                     Cc1ccsc1C(=O)O
Name: 0, dtype: object

In [16]:
metadata.set_index('crystal_name').loc['D68EV3CPROA-x0102_0B']

RealCrystalName                                D68EV3CPROA-x0102
smiles                                            Cc1ccsc1C(=O)O
new_smiles                                        Cc1ccsc1C(=O)O
alternate_name                                        Z104474228
site_name                                                      5
pdb_entry                                                    NaN
mol                <rdkit.Chem.rdchem.Mol object at 0x163d2c200>
single_smiles                                     Cc1ccsc1C(=O)O
Name: D68EV3CPROA-x0102_0B, dtype: object

In [17]:
# NB. This is the Fragalysis download data
# this is for an asymmetric dimer but biological monomer

import pymol2
import chempy # from pymol installation
from pathlib import Path
from typing import List, Set, Dict, Tuple
import json

# ASSUMPTIONS: the PDBs have CONECT records. If not, use connect_mode = 1
# Fragalysis data is fine.
connect_mode = 0 # https://pymolwiki.org/index.php/Connect_mode

# data folder from Fragalysis download
folder = Path('workshop/D68EV3CPROA')
assert folder.exists()
fluff_marker = '§'  # name + fluff + index, say MPRO-x0102_0B§1
reference_chain = 'A'
skip_chains = []  # this is not the target protein, but a partner
lig_resn = 'LIG'

pdb_blocks: Dict[str, str] = {}
details: Dict[str, dict] = {}
for holo in (folder / Path('aligned')).glob('*/*.pdb'):
    #print(holo)
    with pymol2.PyMOL() as pymol:
        pymol.cmd.set('connect_mode',connect_mode)
        pymol.cmd.load('reference.pdb', 'ref')
        pymol_name = 'holo'
        pymol.cmd.load(str(holo.absolute()), pymol_name)
        # get info
        xstal_name: str = holo.name.rstrip('.pdb').rstrip('_bound').strip()
        xstal_info: pd.Series = metadata.set_index('crystal_name').loc[xstal_name]
        # ## Determine what is unique ligand residue
        # get atoms of ligand
        lig_atoms: List[chempy.Atom] = pymol.cmd.get_model(f'%{pymol_name} and resn {lig_resn}').atom
        # get set of tuple of resi chain segi alt
        selectors: Set[tuple] = {(atom.resi, atom.chain, atom.segi, atom.alt) for atom in lig_atoms}
        # iterate for all unique
        i = 0
        for resi, chain, segi, alt in selectors:
            sele = f'%{pymol_name} and resn {lig_resn} and resi {resi}'
            if chain:
                sele+= f' and chain {chain}'
            if segi:
                sele+= f' and segi {segi}'
            if alt:
                sele+= f' and alt {alt}'
            # figure out if it's in the wrong chain (asymmetric dimer)
            n_neigh = pymol.cmd.select('neighs', f'%{pymol_name} and polymer.protein and (byres {sele} around 3)')
            if not n_neigh:
                continue
            polymer_chains = pymol.cmd.get_chains('neighs')
            for polymer_chain in polymer_chains:
                if polymer_chain in skip_chains:
                    continue
                i += 1
                neoname = f'{xstal_name}{fluff_marker}{i}'
                pymol.cmd.align(f'%{pymol_name} and polymer.protein and chain {polymer_chain}',
                                 f'%ref and polymer.protein and chain {reference_chain}')
                pymol.cmd.create('copied', sele)
                pymol.cmd.alter('%copied', 'alt=""')
                pymol.cmd.alter('%copied', 'segi=""')
                pymol.cmd.alter('%copied', 'chain="X"')
                pymol.cmd.alter('%copied', 'resi="1"')
                pymol.cmd.sort()
                pdb_blocks[neoname] = pymol.cmd.get_pdbstr('%copied')
                pymol.cmd.delete('copied')
                details[neoname] = {**xstal_info.to_dict(),
                                    **dict(base_name=xstal_name,
                                        lig_resn=lig_resn,
                                         lig_resi=resi,
                                         lig_chain=chain,
                                         lig_segi=segi,
                                         lig_alt=alt,
                                         polymer_chain=polymer_chain,)
                                    }
            #pymol.cmd.create('modded', sele)
            #print(i, sele, n_neigh)

print('done')

done


In [1]:
import json
import pickle
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

smiles_col_name = 'single_smiles'

mols = []
baddies = []
for name, block in pdb_blocks.items():
    detail: dict = details[name]
    mol = Chem.MolFromPDBBlock(block, proximityBonding='CONECT' not in block)
    if mol is None:
        print(f'Issue with sanitisation, trying without for {name}')
        mol = Chem.MolFromPDBBlock(block, proximityBonding='CONECT' not in block, sanitize=False)
    assert mol, f'{name} failed to load'
    assert mol.GetNumAtoms() > 0 and mol.GetNumBonds() > 0, f'{name} failed to load'
    ref = Chem.MolFromSmiles(detail[smiles_col_name])
    try:
        mol = AllChem.AssignBondOrdersFromTemplate(ref, mol)
    except Exception as e:
        print(f'Issue with bond orders for {name} - {e.__class__.__name__}{e}')
        baddies.append(dict(name=name, block=block, mol=mol, ref=ref, detail=detail, exception=e))
        continue
    mol.SetProp('XChem_code', name.split('_')[0])
    mol.SetProp('Occupancy', json.dumps([a.GetPDBResidueInfo().GetOccupancy() for a in mol.GetAtoms()]) )
    mol.SetProp('TempFactor', json.dumps([a.GetPDBResidueInfo().GetTempFactor() for a in mol.GetAtoms()]) )
    mol.SetProp('_Name', name)
    for k, v in detail.items():
        if isinstance(v, dict):
            v = json.dumps(v)
        mol.SetProp(k, str(v))
    for atom in mol.GetAtoms():
        name = atom.GetPDBResidueInfo().GetName()
        atom.SetProp('molFileAlias', name)
    mols.append(mol)

assert mols, 'No mols!'

Draw.MolsToGridImage(mols)

NameError: name 'pdb_blocks' is not defined

In [36]:
baddies

[{'name': 'D68EV3CPROA-x1064_0B§1',
  'block': 'CRYST1   42.744   62.500  147.471  90.00  90.00  90.00 P 21 21 21    1\nHETATM    1  N   LIG X   1      -2.943   4.236 -24.581  0.60 40.50           N  \nHETATM    2  C   LIG X   1      -0.337   3.794 -25.267  0.60 39.08           C  \nHETATM    3  O   LIG X   1      -1.498   6.099 -25.214  0.60 44.55           O  \nHETATM    4  C1  LIG X   1       0.175   3.013 -26.292  0.60 37.78           C  \nHETATM    5  O1  LIG X   1      -2.242   4.453 -26.911  0.60 42.75           O  \nHETATM    6  C2  LIG X   1       1.246   2.175 -26.034  0.60 37.25           C  \nHETATM    7  O2  LIG X   1      -0.344   4.560 -23.042  0.60 37.51           O  \nHETATM    8  C3  LIG X   1       1.814   2.132 -24.778  0.60 36.67           C  \nHETATM    9  C4  LIG X   1       1.323   2.927 -23.755  0.60 35.55           C  \nHETATM   10  C5  LIG X   1       0.242   3.764 -23.991  0.60 38.41           C  \nHETATM   11  C6  LIG X   1       0.426   5.381 -22.372  0.60

In [ ]:
# error corrections based on failure of below...
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True


# 6 & 12 should not be bondd
name = 'D68EV3CPROA-x1064_0B§1'
block = pdb_blocks[name]